In [1]:
import os
from dotenv import load_dotenv

import pymysql
from sshtunnel import SSHTunnelForwarder

import pandas as pd

In [2]:
load_dotenv()

ssh_username = os.getenv('SSH_USERNAME')
ssh_pkey = os.getenv('SSH_PKEY')
user_name = os.getenv('USER_NAME')
passwd = os.getenv('PASSWD')
db_name = os.getenv('DB_NAME')

In [3]:
query = """
SELECT id, title, content, create_date
FROM portal_news_scraper.portal_news 
ORDER BY create_date DESC
LIMIT 300000;
"""

In [4]:
with SSHTunnelForwarder(
    ("220.118.147.58", 8122), 
    ssh_username=ssh_username, 
    ssh_pkey=ssh_pkey,
    remote_bind_address=("172.30.1.100", 3306), 
) as tunnel:
    print('connect success')
    db = pymysql.connect(
        host='127.0.0.1', 
        port=tunnel.local_bind_port, 
        user=user_name, 
        passwd=passwd, 
        db=db_name
    )
    cursor = db.cursor(pymysql.cursors.DictCursor)
    
    cursor.execute(query)
    data = pd.DataFrame(cursor.fetchall())
    
    cursor.close()
    db.close()

connect success


In [5]:
data.to_pickle('./data/20230413_300000_from_db.pickle')